In [16]:
#食べログから店舗の外観画像と住所をスクレイピング
import requests
from bs4 import BeautifulSoup
import re       
import pandas as pd
import time             
import os           

output_dir="../signboard_classifier/osaka_img/osaka_all_img"  

class Tabelog:

    def __init__(self, base_url, test_mode=True, p_ward='大阪府内', begin_page=1, end_page=15):

        self.store_id = ''
        self.store_id_num = 0
        self.store_name = ''
        self.score = 0
        self.ward = p_ward
        self.review_cnt = 0
        self.review = ''
        self.columns = ['store_id', 'store_name', 'score', 'ward', 'review_cnt', 'review']
        self.df = pd.DataFramecolumns=self.columns)
        self.__regexcomp = re.compile(r'\n|\s')

        page_num = begin_page

        if test_mode:         #test_mode：最初の3店舗を取得
            list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1&freecall=0&LstRange=SF' 
            self.scrape_list(list_url, mode=test_mode)  
        else:
            while True:
                list_url = base_url + str(page_num) +  "/?Srt=D&SrtT=rt&sort_mode=1&freecall=0&LstRange=SF" #食べログの点数ランキングでソートする際に必要な処理
                if self.scrape_list(list_url, mode=test_mode) != True:
                    break        

                if page_num >= end_page:
                    break
                page_num += 1
        return

    
    def scrape_list(self, list_url, mode):
        time.sleep(3)    
        r = requests.get(list_url)       
        if r.status_code != requests.codes.ok:       
            return False          

        soup = BeautifulSoup(r.content, 'html.parser') 
        soup_a_list = soup.find_all('a', class_='list-rst__rst-name-target') 
        
        if len(soup_a_list) == 0:   
            return False
        
        

        if mode:       
            for soup_a in soup_a_list[:3]:  
                item_url = soup_a.get('href')+'/dtlphotolst/4/smp2/D-normal'
                self.store_id_num += 1      
                self.scrape_item(item_url, mode) 
                print(item_url) 
                
                map_url = soup_a.get('href')+'/dtlmap/' 
                self.scrape_address(map_url, mode) 
              
                
        else:
            for soup_a in soup_a_list:
                #各店舗の外観画像ページURLの取得
                item_url = soup_a.get('href')+'/dtlphotolst/4/smp2/D-normal'
                self.store_id_num += 1 
                self.scrape_item(item_url, mode)
                print(item_url)   
                
                #各店舗の住所の取得 
                map_url = soup_a.get('href')+'/dtlmap/'         
                self.scrape_address(map_url, mode)  
                
        return True      

    
    
    #各店舗の外観画像を取得
    def scrape_item(self, item_url, mode):
        
        start = time.time()  
        home_url=item_url.rsplit("/",5)[0] 
        label=home_url.split("/",3)[3] 
        label=label.replace("/","_")  
        
        for page in range(1,21): 
            time.sleep(3)    
            r = requests.get(iem_url+"/"+str(page)) 
            if r.status_code != requests.codes.ok:    
                print(f'error:not found{ item_url+"/"+str(page) }')     
                return    

            soup = BeautifulSoup(r.content, 'html.parser')         
            img_tag_list = soup.find_all('a',class_='js-imagebox-trigger js-analytics rstdtl-thumb-list__target')
        
            for img_tag in img_tag_list:   
                img_url = img_tag.get('href')  
                time.sleep(3)   
                img = requests.get(img_url)   
                if not os.path.exists(output_dir+label):  
                    os.makedirs(output_dir+label)
                with open(output_dir+label+'/'+img_url.split('/')[-1],'wb') as f: 
                    f.write(img.content)
                print(img_url)  
        return
  
                      
        
    #各店舗の住所を取得                      
    def scrape_address(self, map_url, mode):      
                      
        start = time.time()    
        home_url=map_url.rsplit("/",3)[0] 
        label=home_url.split("/",3)[3] 
        label=label.replace("/","_")  
                      
        r = requests.get(map_url) 
                    
        if r.status_code != requests.codes.ok:          
            return False                  
                      
        soup = BeautifulSoup(r.text,"html.parser")
        tables = soup.find('table',class_="c-table c-table--bordered")                         
        rows = tables.findAll("td")  
        print(rows[0].get_text())          
                      
        if not os.path.exists(output_dir+label):
            os.makedirs(output_dir+label)
        f = open(output_dir+label + "/" + label + ".txt",'w',encoding="UTF-8")
        f.write(rows[0].get_text())              
        f.close()                      
        return

In [ ]:
Tabelog(base_url="https://tabelog.com/osaka/A2701/A270302/R4893/rstLst/",test_mode=False,end_page=30)

https://tblg.k-img.com/restaurant/images/Rvw/184459/640x640_rect_ffd01e5c4445f3b72623225d20fffe80.jpg
https://tblg.k-img.com/restaurant/images/Rvw/183245/640x640_rect_4f5ecd691cd83b3c8058550b18c1a377.jpg
https://tblg.k-img.com/restaurant/images/Rvw/182727/640x640_rect_c1566ae40e47e559f147acf67d14f079.jpg
https://tblg.k-img.com/restaurant/images/Rvw/182355/640x640_rect_f812e6d6703218ee24dc5337e9fa8b5a.jpg
https://tblg.k-img.com/restaurant/images/Rvw/182355/640x640_rect_b5e456f22cd5c340eb3bd6ccc5cb6e77.jpg
https://tblg.k-img.com/restaurant/images/Rvw/182355/640x640_rect_3c7edc5f2195db7d7db04ed7ed39280b.jpg
https://tblg.k-img.com/restaurant/images/Rvw/180683/640x640_rect_dc526c40252780d333fc34c74c74fb55.jpg
https://tblg.k-img.com/restaurant/images/Rvw/180137/640x640_rect_348fa90b04ed070d07e6f19a264dfe33.jpg
https://tblg.k-img.com/restaurant/images/Rvw/178104/640x640_rect_d5b43c8a03c846cf623a4a9d66fee477.jpg
https://tblg.k-img.com/restaurant/images/Rvw/178075/640x640_rect_7d90953751f0691ea

https://tblg.k-img.com/restaurant/images/Rvw/132485/640x640_rect_132485875.jpg
https://tblg.k-img.com/restaurant/images/Rvw/128987/640x640_rect_128987011.jpg
https://tblg.k-img.com/restaurant/images/Rvw/127943/640x640_rect_127943691.jpg
https://tblg.k-img.com/restaurant/images/Rvw/127609/640x640_rect_127609593.jpg
https://tblg.k-img.com/restaurant/images/Rvw/125946/640x640_rect_125946985.jpg
https://tblg.k-img.com/restaurant/images/Rvw/125135/640x640_rect_125135020.jpg
https://tblg.k-img.com/restaurant/images/Rvw/125135/640x640_rect_125135025.jpg
https://tblg.k-img.com/restaurant/images/Rvw/125135/640x640_rect_125135041.jpg
https://tblg.k-img.com/restaurant/images/Rvw/125135/640x640_rect_125135043.jpg
https://tblg.k-img.com/restaurant/images/Rvw/124963/640x640_rect_124963107.jpg
https://tblg.k-img.com/restaurant/images/Rvw/124706/640x640_rect_124706038.jpg
https://tblg.k-img.com/restaurant/images/Rvw/124279/640x640_rect_124279450.jpg
https://tblg.k-img.com/restaurant/images/Rvw/122960/

https://tblg.k-img.com/restaurant/images/Rvw/180488/640x640_rect_4cc1a01e6990f023edfabbc7d9b945bd.jpg
https://tblg.k-img.com/restaurant/images/Rvw/179501/640x640_rect_d91f4ece2e1b8ab0728b0b92238bf1c0.jpg
https://tblg.k-img.com/restaurant/images/Rvw/179945/640x640_rect_d91f4ece2e1b8ab0728b0b92238bf1c0.jpg
https://tblg.k-img.com/restaurant/images/Rvw/180160/640x640_rect_d80010bb7f11ee9095c95c3f90430860.jpg
https://tblg.k-img.com/restaurant/images/Rvw/180160/640x640_rect_d80010bb7f11ee9095c95c3f90430860.jpg
https://tblg.k-img.com/restaurant/images/Rvw/178971/640x640_rect_da0d28cc299f0c6974ba4ac3ff946601.jpg
https://tblg.k-img.com/restaurant/images/Rvw/178367/640x640_rect_818db65bbd9ea1258cfbb35c2c4f5a15.jpg
https://tblg.k-img.com/restaurant/images/Rvw/178202/640x640_rect_8f834ae74536c7bb142380a6c13cc956.jpg
https://tblg.k-img.com/restaurant/images/Rvw/178181/640x640_rect_d6e34bbbb15262cc8e40e0d2363ab6eb.jpg
https://tblg.k-img.com/restaurant/images/Rvw/176197/640x640_rect_dfdfe796cc99b2086

https://tblg.k-img.com/restaurant/images/Rvw/105967/105967058.jpg
https://tblg.k-img.com/restaurant/images/Rvw/105126/640x640_rect_105126449.jpg
https://tblg.k-img.com/restaurant/images/Rvw/103432/640x640_rect_103432096.jpg
https://tblg.k-img.com/restaurant/images/Rvw/101494/640x640_rect_101494593.jpg
https://tblg.k-img.com/restaurant/images/Rvw/102050/640x640_rect_102050166.jpg
https://tblg.k-img.com/restaurant/images/Rvw/100877/640x640_rect_100877322.jpg
https://tblg.k-img.com/restaurant/images/Rvw/100905/640x640_rect_100905599.jpg
https://tblg.k-img.com/restaurant/images/Rvw/135322/640x640_rect_135322164.jpg
https://tblg.k-img.com/restaurant/images/Rvw/97534/640x640_rect_97534889.jpg
https://tblg.k-img.com/restaurant/images/Rvw/96010/640x640_rect_96010748.jpg
https://tblg.k-img.com/restaurant/images/Rvw/96010/640x640_rect_96010751.jpg
https://tblg.k-img.com/restaurant/images/Rvw/93135/640x640_rect_93135616.jpg
https://tblg.k-img.com/restaurant/images/Rvw/92783/640x640_rect_92783826.

https://tblg.k-img.com/restaurant/images/Rvw/130798/640x640_rect_130798555.jpg
https://tblg.k-img.com/restaurant/images/Rvw/130837/640x640_rect_130837623.jpg
https://tblg.k-img.com/restaurant/images/Rvw/124378/640x640_rect_124378877.jpg
https://tblg.k-img.com/restaurant/images/Rvw/123035/640x640_rect_123035227.jpg
https://tblg.k-img.com/restaurant/images/Rvw/121273/640x640_rect_121273928.jpg
https://tblg.k-img.com/restaurant/images/Rvw/114806/640x640_rect_114806083.jpg
https://tblg.k-img.com/restaurant/images/Rvw/112594/640x640_rect_112594827.jpg
https://tblg.k-img.com/restaurant/images/Rvw/108341/640x640_rect_108341353.jpg
https://tblg.k-img.com/restaurant/images/Rvw/104032/640x640_rect_104032844.jpg
https://tblg.k-img.com/restaurant/images/Rvw/100159/640x640_rect_100159718.jpg
https://tblg.k-img.com/restaurant/images/Rvw/100031/640x640_rect_100031418.jpg
https://tblg.k-img.com/restaurant/images/Rvw/99929/99929864.jpg
https://tblg.k-img.com/restaurant/images/Rvw/92976/640x640_rect_929

...

https://tblg.k-img.com/restaurant/images/Rvw/116320/640x640_rect_116320400.jpg
https://tblg.k-img.com/restaurant/images/Rvw/96752/640x640_rect_96752545.jpg
https://tblg.k-img.com/restaurant/images/Rvw/92764/640x640_rect_92764902.jpg
https://tblg.k-img.com/restaurant/images/Rvw/92552/640x640_rect_92552840.jpg
https://tblg.k-img.com/restaurant/images/Rvw/86625/640x640_rect_86625321.jpg
https://tblg.k-img.com/restaurant/images/Rvw/86624/640x640_rect_86624875.jpg
https://tblg.k-img.com/restaurant/images/Rvw/85755/640x640_rect_85755328.jpg
https://tblg.k-img.com/restaurant/images/Rvw/55702/640x640_rect_55702844.jpg
https://tblg.k-img.com/restaurant/images/Rvw/55702/640x640_rect_55702840.jpg
https://tblg.k-img.com/restaurant/images/Rvw/13853/640x640_rect_13853890.jpg
https://tblg.k-img.com/restaurant/images/Rvw/1024/1024385.jpg
error:not foundhttps://tabelog.com/osaka/A2701/A270302/27000977//dtlphotolst/4/smp2/D-normal/2
https://tabelog.com/osaka/A2701/A270302/27000977//dtlphotolst/4/smp2/D-

https://tblg.k-img.com/restaurant/images/Rvw/109693/640x640_rect_109693703.jpg
https://tblg.k-img.com/restaurant/images/Rvw/79279/640x640_rect_79279276.jpg
https://tblg.k-img.com/restaurant/images/Rvw/65703/640x640_rect_65703712.jpg
https://tblg.k-img.com/restaurant/images/Rvw/51007/640x640_rect_51007594.jpg
https://tblg.k-img.com/restaurant/images/Rvw/51007/640x640_rect_51007591.jpg
https://tblg.k-img.com/restaurant/images/Rvw/162722/640x640_rect_162722388.jpg
https://tblg.k-img.com/restaurant/images/Rvw/157790/640x640_rect_157790875.jpg
https://tblg.k-img.com/restaurant/images/Rvw/157790/640x640_rect_157790877.jpg
https://tblg.k-img.com/restaurant/images/Rvw/157790/640x640_rect_157790894.jpg
https://tblg.k-img.com/restaurant/images/Rvw/151229/640x640_rect_151229456.jpg
https://tblg.k-img.com/restaurant/images/Rvw/135282/640x640_rect_135282723.jpg
https://tblg.k-img.com/restaurant/images/Rvw/128263/640x640_rect_128263234.jpg
https://tblg.k-img.com/restaurant/images/Rvw/127077/640x640_


                  大阪府
                  大阪市淀川区
                    十三本町
                  2-1-17
                  ナポリ十三ビル 1F              
https://tblg.k-img.com/restaurant/images/Rvw/113513/640x640_rect_113513714.jpg
https://tblg.k-img.com/restaurant/images/Rvw/67598/640x640_rect_67598693.jpg
https://tblg.k-img.com/restaurant/images/Rvw/102630/640x640_rect_102630439.jpg
https://tblg.k-img.com/restaurant/images/Rvw/130273/640x640_rect_130273545.jpg
https://tblg.k-img.com/restaurant/images/Rvw/130273/130273546.jpg
https://tblg.k-img.com/restaurant/images/Rvw/130273/640x640_rect_130273605.jpg
https://tblg.k-img.com/restaurant/images/Rvw/109698/640x640_rect_109698870.jpg
https://tblg.k-img.com/restaurant/images/Rvw/109698/640x640_rect_109698875.jpg
https://tblg.k-img.com/restaurant/images/Rvw/76331/76331309.jpg
https://tblg.k-img.com/restaurant/images/Rvw/76331/76331318.jpg
https://tblg.k-img.com/restaurant/images/Rvw/66313/66313661.jpg
error:not foundhttps://tabelog.com/osaka/A2701/A27

https://tblg.k-img.com/restaurant/images/Rvw/141913/640x640_rect_141913917.jpg
https://tblg.k-img.com/restaurant/images/Rvw/141913/640x640_rect_141913918.jpg
https://tblg.k-img.com/restaurant/images/Rvw/141923/640x640_rect_141923271.jpg
https://tblg.k-img.com/restaurant/images/Rvw/136378/136378721.jpg
https://tblg.k-img.com/restaurant/images/Rvw/136184/136184819.jpg
https://tblg.k-img.com/restaurant/images/Rvw/133044/640x640_rect_133044724.jpg
https://tblg.k-img.com/restaurant/images/Rvw/133044/640x640_rect_133044735.jpg
https://tblg.k-img.com/restaurant/images/Rvw/119626/640x640_rect_119626672.jpg
https://tblg.k-img.com/restaurant/images/Rvw/119513/640x640_rect_119513348.jpg
https://tblg.k-img.com/restaurant/images/Rvw/113939/640x640_rect_113939553.jpg
https://tblg.k-img.com/restaurant/images/Rvw/141747/640x640_rect_141747681.jpg
https://tblg.k-img.com/restaurant/images/Rvw/141747/640x640_rect_141747682.jpg
https://tblg.k-img.com/restaurant/images/Rvw/141747/640x640_rect_141747683.jpg

error:not foundhttps://tabelog.com/osaka/A2701/A270302/27110096//dtlphotolst/4/smp2/D-normal/2
https://tabelog.com/osaka/A2701/A270302/27110096//dtlphotolst/4/smp2/D-normal

                  大阪府
                  大阪市淀川区
                    十三東
                  2-11-14
              
https://tblg.k-img.com/restaurant/images/Rvw/110533/640x640_rect_110533337.jpg
https://tblg.k-img.com/restaurant/images/Rvw/110533/640x640_rect_110533328.jpg
https://tblg.k-img.com/restaurant/images/Rvw/110533/640x640_rect_110533318.jpg
https://tblg.k-img.com/restaurant/images/Rvw/132556/640x640_rect_132556811.jpg
https://tblg.k-img.com/restaurant/images/Rvw/126428/640x640_rect_126428811.jpg
https://tblg.k-img.com/restaurant/images/Rvw/116451/640x640_rect_116451799.jpg
https://tblg.k-img.com/restaurant/images/Rvw/122730/640x640_rect_122730992.jpg
https://tblg.k-img.com/restaurant/images/Rvw/116564/116564460.jpg
https://tblg.k-img.com/restaurant/images/Rvw/113784/640x640_rect_113784306.jpg
https://tblg.k-i